# Players by Location
- create a map using folium that plots the hometown location of every US (and maybe Canadian) player in D1 (2024-25 season)

In [29]:
## Dependencies

## System Libraries
import sys
import os
# Data handling
import pandas as pd
import geopandas as gpd
# Plotting and visualization
import matplotlib.pyplot as plt
from PIL import Image
## Map visualization
import folium
from folium.plugins import MarkerCluster
from folium.plugins import HeatMap

### ROSTERFILE
# Path to the roster file
roster_path = os.path.join('..', 'data', 'roster_2024_current_v3.csv')
roster_df = pd.read_csv(roster_path) # Load the roster file

### SHAPEFILES
# Path to .geojson file with State Boundries
geojson_path = os.path.join('..', 'data', 'vault', 'States_shapefile.shp')
# Load the states shapefile
gdf_states = gpd.read_file(geojson_path)

# Path to shapefile with all US counties
shapefile_path = os.path.join('..', 'data', 'vault', 'cb_2018_us_county_500k.shp')
gdf = gpd.read_file(shapefile_path)
# Set the initial CRS (assuming it's in EPSG:4326, but you may need to verify the original CRS)
gdf = gdf.set_crs(epsg=4326)

## CHECK SHAPEFILES FOR COMPATIBILITY
# Set the CRS for both dataframes if it's missing
if gdf.crs is None:
    gdf.set_crs(epsg=4326, inplace=True)  # Assuming coordinates are in WGS 84 (lat/lon)

if gdf_states.crs is None:
    gdf_states.set_crs(epsg=4326, inplace=True)  # Assuming coordinates are in WGS 84 (lat/lon)


# Check the first few rows of the DataFrames
# roster_df.head()
# gdf_states.head()
# gdf.head()



## Prep The Data

In [30]:
# Clean up and prepare the data for mapping

# Drop rows with missing geographic data
geo_columns = ['City', 'State_Province', 'Country']
roster_cleaned_df = roster_df.dropna(subset=geo_columns)

# Create a simplified dataframe with relevant location columns for easier mapping
roster_cleaned_df = roster_cleaned_df[['Current Team', 'First_Name', 'Last_Name', 'City', 'State_Province', 'Country']]

# Group by City, State_Province, and Country to count the number of players from each location
location_counts = roster_cleaned_df.groupby(['City', 'State_Province', 'Country']).size().reset_index(name='Player_Count')


## CHECK DATA TRANSFORMATION
# # Display the cleaned and grouped dataframe
# location_counts.head()
# # Sort the location counts in descending order
# location_counts_sorted = location_counts.sort_values(by='Player_Count', ascending=False)
# location_counts_sorted.head(10) # Display


### Geocode Conversion
- takes the names of places and converts to lat long coordinates
- uses a rate limiter to avoid overloading service
- takes about 15 min to run - output is saved in the data folder - load from there 

In [31]:
## LCHECK FOR AND LOAD GEOCODED DATA BEFORE RUNNING - THIS TAKES 15+ MINUTES

# from geopy.geocoders import Nominatim
# from geopy.extra.rate_limiter import RateLimiter

# # Initialize geocoder
# geolocator = Nominatim(user_agent="college_hockey_map")

# # Create a rate-limited geocode function to avoid overloading the service
# geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

# # Function to geocode a city and state combination
# def geocode_location(row):
#     try:
#         location_str = f"{row['City']}, {row['State_Province']}, {row['Country']}"
#         location = geocode(location_str)
#         if location:
#             return pd.Series([location.latitude, location.longitude])
#         else:
#             return pd.Series([None, None])
#     except Exception as e:
#         return pd.Series([None, None])

# # Apply the geocode function to the data
# location_counts[['Latitude', 'Longitude']] = location_counts.apply(geocode_location, axis=1)

# # Filter out rows with missing coordinates
# location_counts_cleaned = location_counts.dropna(subset=['Latitude', 'Longitude'])

# # Display cleaned data with coordinates
# location_counts_cleaned.head()


In [32]:
## Save Geocoded Data to CSV to avoid having to run geocoding repeatedly

# Save the cleaned and geocoded data to a CSV file
output_path = os.path.join('..', 'data', 'player_geocoded_location_counts.csv')
location_counts_cleaned.to_csv(output_path, index=False)

#### Load the previously geocoded table from csv

In [33]:
## Path to the geocoded data
geocoded_data_path = os.path.join('..', 'data', 'player_geocoded_location_counts.csv')
location_counts_cleaned = pd.read_csv(geocoded_data_path)

# Check the first few rows of the geocoded data
location_counts_cleaned.head()

,City,State_Province,Country,Player_Count,Latitude,Longitude
0,Abbotsford,British Columbia,Canada,4,49.052116,-122.329479
1,Abington,Massachusetts,USA,1,42.104823,-70.945322
2,Airdrie,Alberta,Canada,3,51.285970,-114.010620
3,Albert Lea,Minnesota,USA,1,43.648013,-93.368266
4,Alexandria,Minnesota,USA,1,45.883510,-95.374501


### Add Player name, team, ect data into the location counts

In [34]:
# Merge roster_df with location_counts_cleaned
merged_df = pd.merge(roster_df, location_counts_cleaned, 
                     on=['City', 'State_Province', 'Country'], how='inner')

# Prepare the tooltip text for each location
merged_df['Tooltip'] = merged_df.apply(
    lambda row: f"{row['First_Name']} {row['Last_Name']} - {row['Position']} - {row['Yr']} - {row['Current Team']}", axis=1
)

# Display the merged dataframe to check if the tooltips are correctly generated
merged_df[['First_Name', 'Last_Name', 'Tooltip', 'Latitude', 'Longitude']].head()

,First_Name,Last_Name,Tooltip,Latitude,Longitude
0,Adam,Barone,Adam Barone - Defensemen - Fr - Lake Superior,46.523910,-84.320068
1,Jack,Blanchett,Jack Blanchett - Defensemen - So - Lake Superior,41.915336,-83.513566
2,Mike,Brown,Mike Brown - Defensemen - Jr - Lake Superior,42.395930,-71.178665
3,Evan,Bushy,Evan Bushy - Defensemen - So - Lake Superior,44.163466,-93.999351
4,Jacob,Conrad,Jacob Conrad - Defensemen - Fr - Lake Superior,44.512638,-88.012579


## Start Mapping

#### Very Simple V0.1 Map

In [35]:

# Function to create the player origin map with tooltips
def create_player_origin_map_with_tooltip(data, map_center=[45.0, -93.0], zoom_start=4):
    # Map settings block for customization
    folium_map = folium.Map(location=map_center, zoom_start=zoom_start, tiles='cartodb positron')
    
    # Create MarkerCluster
    marker_cluster = MarkerCluster().add_to(folium_map)

    # Add player markers with tooltips to the MarkerCluster
    for _, row in data.iterrows():
        folium.Marker(
            location=[row['Latitude'], row['Longitude']],
            tooltip=row['Tooltip']
        ).add_to(marker_cluster)
    
    return folium_map

# Create the player origins map with tooltips
player_map = create_player_origin_map_with_tooltip(merged_df)

# # Save the map to an HTML file for visualization
# map_file_path = os.path.join('..', 'TEMP', 'player_origin_map_v1.html')
# player_map.save(map_file_path)

# map_file_path


#### Aggrigate data on state by state basis 
- will use to color specific states - regions

In [36]:
# First, let's aggregate the player counts by State_Province and Country for the choropleth layer
player_counts_by_region = merged_df.groupby(['State_Province', 'Country']).agg(
    {'Player_Count': 'sum', 'Latitude': 'mean', 'Longitude': 'mean'}).reset_index()

# Check the aggregated data for State/Province player counts
player_counts_by_region.head(20)


,State_Province,Country,Player_Count,Latitude,Longitude
0,Alaska,USA,137,61.914700,-149.386868
1,Alberta,Canada,2658,51.864160,-113.741660
2,Arizona,USA,28,33.458629,-111.955474
3,Belarus,Belarus,4,53.917291,27.544940
4,British Columbia,Canada,513,49.486323,-122.734645
5,California,USA,200,34.380970,-118.695080
6,Colorado,USA,66,39.637425,-105.150913
7,Connecticut,USA,86,41.280479,-73.233813
8,Delaware,USA,1,39.745947,-75.546589
9,District of Columbia,USA,1,38.895037,-77.036543


# Stopped Here Thursday Night
### Code below is the example of next steps from GPI
#### To-DO
- get geodata (state-province boundries, possibly also another layer - county, ect)
- GEOJSON format

In [ ]:
############ EAMPLE CODE

import json
import geopandas as gpd
from folium.plugins import HeatMap

# Load a world GeoJSON file (assuming we're focusing on Canada/USA for now) to apply fill colors to states/provinces
# Unfortunately, I cannot access a remote GeoJSON file, but you can use one locally, for example:
# geojson_path = 'path_to_geojson_file'

# For demonstration, we'll simulate this step

# Now, let's create a map with choropleth and heatmap

def create_choropleth_heatmap_map(data, map_center=[45.0, -93.0], zoom_start=4):
    # Initialize the map
    folium_map = folium.Map(location=map_center, zoom_start=zoom_start, tiles='cartodb positron')
    
    # Heatmap layer
    heat_data = [[row['Latitude'], row['Longitude']] for _, row in data.iterrows()]
    HeatMap(heat_data, radius=12, blur=15, max_intensity=10).add_to(folium_map)
    
    # Assuming we have a GeoJSON file with state/province boundaries, we would use it here
    # folium.Choropleth(
    #     geo_data=geojson_path,  # Provide the path to a geojson file with state/province boundaries
    #     data=player_counts_by_region,
    #     columns=['State_Province', 'Player_Count'],
    #     key_on='feature.properties.NAME',  # Adjust this to match the key in the GeoJSON file
    #     fill_color='YlGnBu',
    #     fill_opacity=0.7,
    #     line_opacity=0.2,
    #     legend_name='Number of Players by State/Province'
    # ).add_to(folium_map)
    
    # Add markers with tooltips as in the previous map
    marker_cluster = MarkerCluster().add_to(folium_map)
    for _, row in data.iterrows():
        folium.Marker(
            location=[row['Latitude'], row['Longitude']],
            tooltip=row['Tooltip']
        ).add_to(marker_cluster)
    
    return folium_map

# Create the enhanced map with choropleth and heatmap
enhanced_player_map = create_choropleth_heatmap_map(merged_df)

# Save the map to an HTML file for visualization
# enhanced_map_file_path = '/mnt/data/college_hockey_player_origin_map_enhanced.html'
enhanced_player_map.save(enhanced_map_file_path)

enhanced_map_file_path


